In [0]:
import time
import random
import json
import os # Import the os module for directory operations

# --- Configuration Parameters ---
# Define distances between locations (in meters)
# This will be dynamically generated based on the number of sources and stockpiles.
DISTANCES = {}

# Truck speeds (meters per second)
TRUCK_SPEED_MPS = 10  # 36 km/h

# Unloading rate (tonnes per second)
UNLOADING_RATE_TPS = 5

# --- Base Classes ---

class Location:
    """Base class for any location in the simulation."""
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

class Source(Location):
    """Represents a material source."""
    def __init__(self, name):
        super().__init__(name)
        # In a more complex simulation, sources might have material quantity, quality, etc.

class Stockpile(Location):
    """Represents a material drop-off location."""
    def __init__(self, name):
        super().__init__(name)
        # In a more complex simulation, stockpiles might track accumulated material.

class Excavator:
    """Represents an excavator that loads material onto trucks."""
    def __init__(self, name, location, loading_rate_tps):
        self.name = name
        self.location = location  # The source it's operating at
        self.loading_rate_tps = loading_rate_tps  # Tonnes per second

    def __str__(self):
        return f"Excavator {self.name} at {self.location.name}"

# --- Dump Truck Class ---

class DumpTruck:
    """Represents a dump truck in the simulation."""
    def __init__(self, truck_id, capacity, initial_source, initial_stockpile):
        self.truck_id = truck_id
        self.capacity = capacity  # Max tonnes the truck can carry
        self.current_tonnes = 0
        self.cycle_counter = 0
        self.current_activity = "IDLE"  # States: IDLE, DRIVING_TO_SOURCE, LOADING, DRIVING_TO_STOCKPILE, UNLOADING
        self.previous_activity = "INITIAL_STATE" # Added to track previous activity for reporting
        self.target_source = initial_source
        self.target_stockpile = initial_stockpile
        self.current_location = initial_stockpile # Start at a stockpile or a neutral point
        self.time_in_current_activity = 0 # Time remaining for current activity

    def _calculate_travel_time(self, start_location, end_location):
        """Calculates travel time between two locations based on DISTANCES."""
        distance_key = (start_location.name, end_location.name)
        distance_key_reversed = (end_location.name, start_location.name)

        distance = DISTANCES.get(distance_key)
        if distance is None:
            distance = DISTANCES.get(distance_key_reversed)

        if distance is None:
            # Fallback for undefined distances, should ideally not happen with dynamic generation
            print(f"Warning: Distance not defined between {start_location.name} and {end_location.name}. Assuming 1000m.")
            distance = 1000 # Default if not found

        return distance / TRUCK_SPEED_MPS

    def update(self, delta_time, excavators):
        """
        Updates the truck's state based on elapsed time.
        Args:
            delta_time (int): The time elapsed in seconds (usually 1 for this simulation).
            excavators (list): List of available excavators.
        """
        self.previous_activity = self.current_activity # Store current activity before potential change
        self.time_in_current_activity -= delta_time

        if self.time_in_current_activity <= 0:
            # Activity completed, transition to next state
            if self.current_activity == "IDLE":
                self.current_activity = "DRIVING_TO_SOURCE"
                self.time_in_current_activity = self._calculate_travel_time(self.current_location, self.target_source)
                self.current_location = None # Truck is in transit

            elif self.current_activity == "DRIVING_TO_SOURCE":
                self.current_activity = "LOADING"
                self.current_location = self.target_source
                # Find an excavator at the current source
                excavator_at_source = next((e for e in excavators if e.location == self.target_source), None)
                if excavator_at_source:
                    self.time_in_current_activity = self.capacity / excavator_at_source.loading_rate_tps
                else:
                    print(f"Error: No excavator found at {self.target_source.name} for Truck {self.truck_id}. Truck stuck.")
                    self.time_in_current_activity = float('inf') # Truck is stuck
                self.cycle_counter += 1 # Cycle increments when loading begins

            elif self.current_activity == "LOADING":
                self.current_tonnes = self.capacity # Truck is now fully loaded
                self.current_activity = "DRIVING_TO_STOCKPILE"
                # For driving from source to stockpile, use the source as the start location
                self.time_in_current_activity = self._calculate_travel_time(self.target_source, self.target_stockpile)
                self.current_location = None # Truck is in transit

            elif self.current_activity == "DRIVING_TO_STOCKPILE":
                self.current_activity = "UNLOADING"
                self.current_location = self.target_stockpile
                self.time_in_current_activity = self.current_tonnes / UNLOADING_RATE_TPS

            elif self.current_activity == "UNLOADING":
                self.current_tonnes = 0 # Truck is now empty
                self.current_activity = "DRIVING_TO_SOURCE"
                # For driving from stockpile back to source, use the stockpile as the start location
                self.time_in_current_activity = self._calculate_travel_time(self.target_stockpile, self.target_source)
                self.current_location = None # Truck is in transit

    def get_report(self):
        """Returns a dictionary detailing the truck's current status."""
        location_name = None
        if self.current_activity == "DRIVING_TO_SOURCE":
            location_name = self.target_source.name
        elif self.current_activity == "DRIVING_TO_STOCKPILE":
            location_name = self.target_stockpile.name
        elif self.current_activity in ["LOADING", "IDLE"]: # IDLE is at a stockpile
            location_name = self.current_location.name
        elif self.current_activity == "UNLOADING":
            location_name = self.current_location.name

        return {
            "truck_id": self.truck_id,
            "activity": self.current_activity.replace('_', ' '),
            "location": location_name,
            "tonnes_carried": round(self.current_tonnes, 1), # Round for cleaner output
            "cycle_count": self.cycle_counter,
            "assigned_source": self.target_source.name, # Added assigned source
            "assigned_stockpile": self.target_stockpile.name # Added assigned stockpile
        }

# --- Simulation Setup Helper Functions ---

def create_sources(num_sources):
    """Creates a list of Source objects."""
    return [Source(f"Source {chr(65 + i)}") for i in range(num_sources)]

def create_stockpiles(num_stockpiles):
    """Creates a list of Stockpile objects."""
    return [Stockpile(f"Stockpile {i + 1}") for i in range(num_stockpiles)]

def create_excavators(num_excavators, sources):
    """
    Creates a list of Excavator objects and assigns them to sources.
    Excavators are distributed evenly among available sources.
    """
    excavators = []
    if not sources:
        print("Warning: No sources available to assign excavators.")
        return []
    for i in range(num_excavators):
        assigned_source = sources[i % len(sources)] # Distribute excavators among sources
        excavators.append(Excavator(f"E{i+1}", assigned_source, loading_rate_tps=random.uniform(2, 3)))
    return excavators

def create_trucks(num_trucks, sources, stockpiles):
    """
    Creates a list of DumpTruck objects, assigning them random capacities
    and distributing them among available sources and stockpiles.
    """
    trucks = []
    if not sources or not stockpiles:
        print("Error: Cannot create trucks without sources or stockpiles.")
        return []
    for i in range(num_trucks):
        source = sources[i % len(sources)] # Cycle through sources
        stockpile = stockpiles[i % len(stockpiles)] # Cycle through stockpiles
        capacity = random.randint(80, 120)  # Random capacity between 80 and 120 tonnes
        trucks.append(DumpTruck(f"T{i+1}", capacity=capacity, initial_source=source, initial_stockpile=stockpile))
    return trucks

def generate_distances(sources, stockpiles):
    """
    Dynamically generates distances between all source-stockpile pairs.
    Populates the global DISTANCES dictionary.
    """
    global DISTANCES
    DISTANCES = {}
    for source in sources:
        for stockpile in stockpiles:
            # Assign a random distance between 4000m and 8000m
            distance = random.randint(4000, 8000)
            DISTANCES[(source.name, stockpile.name)] = distance
            DISTANCES[(stockpile.name, source.name)] = distance # Add reverse path

# --- Simulation Setup ---

def run_simulation(duration_seconds, num_trucks, num_excavators, num_sources, num_stockpiles):
    """
    Runs the mining simulation for a specified duration with configurable assets.
    Args:
        duration_seconds (int): The total time to run the simulation in seconds.
        num_trucks (int): Number of dump trucks to simulate.
        num_excavators (int): Number of excavators to simulate.
        num_sources (int): Number of material sources.
        num_stockpiles (int): Number of stockpiles (destinations).
    """
    print(f"--- Starting Mining Simulation for {duration_seconds} seconds ---")
    print(f"Configuration: {num_trucks} trucks, {num_excavators} excavators, "
          f"{num_sources} sources, {num_stockpiles} stockpiles.")

    # 1. Define Locations
    sources = create_sources(num_sources)
    stockpiles = create_stockpiles(num_stockpiles)

    # Generate distances based on the created locations
    generate_distances(sources, stockpiles)

    # 2. Define Excavators
    excavators = create_excavators(num_excavators, sources)

    # 3. Define Dump Trucks
    trucks = create_trucks(num_trucks, sources, stockpiles)

    # Initialize trucks to start their first activity
    for truck in trucks:
        truck.current_activity = "IDLE" # Set to IDLE to trigger initial DRIVING_TO_SOURCE
        truck.time_in_current_activity = 0 # Ensures they start immediately
        truck.previous_activity = "INITIAL_STATE" # Ensure initial print

    # Create a directory for reports if it doesn't exist
    reports_dir = "truck_reports"
    os.makedirs(reports_dir, exist_ok=True)
    print(f"Saving reports to: {os.path.abspath(reports_dir)}")

    # --- Simulation Loop ---
    for current_time in range(duration_seconds):
        reports_this_second = []
        for truck in trucks:
            truck.update(delta_time=1, excavators=excavators)
            if truck.current_activity != truck.previous_activity:
                reports_this_second.append(truck.get_report())

        if reports_this_second:
            print(f"\n--- Time: {current_time + 1}s ---")
            for report in reports_this_second:
                # Print to console
                print(json.dumps(report, indent=2))

                # Save to file
                filename = f"time_{current_time + 1}_truck_{report['truck_id']}.json"
                filepath = os.path.join(reports_dir, filename)
                with open(filepath, 'w') as f:
                    json.dump(report, f, indent=2)
                # print(f"Saved: {filepath}") # Uncomment to see file save confirmations

        time.sleep(0.1) # Added a small delay to visualize the simulation in real-time

    print("\n--- Simulation Finished ---")

# --- Run the Simulation ---
if __name__ == "__main__":
    try:
        sim_duration = 3000
        num_trucks = 10
        num_excavators = 6
        num_sources = num_excavators
        num_stockpiles = 12

        if num_sources <= 0 or num_stockpiles <= 0:
            print("Error: Number of sources and stockpiles must be greater than 0.")
        elif num_excavators <= 0:
            print("Error: Number of excavators must be greater than 0.")
        elif num_trucks <= 0:
            print("Error: Number of trucks must be greater than 0.")
        else:
            run_simulation(sim_duration, num_trucks, num_excavators, num_sources, num_stockpiles)
    except ValueError:
        print("Invalid input. Please enter integer values.")
    except Exception as e:
        print(f"An error occurred: {e}")
